In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
customers = pd.read_csv("data/customers.csv")
customers.head(5)

,customer_id,customer_name,age,gender
0,1,Customer 1,25,Male
1,2,Customer 2,40,Female
2,3,Customer 3,30,Male
3,4,Customer 4,33,Male
4,5,Customer 5,43,Male


In [4]:
products = pd.read_csv("data/products.csv")
products.head(5)

,product_id,product_name,category,price
0,1,Product 1,Books,14.02
1,2,Product 2,Books,291.83
2,3,Product 3,Books,295.06
3,4,Product 4,Electronics,228.15
4,5,Product 5,Furniture,419.20


In [5]:
stores = pd.read_csv("data/stores.csv")
stores.head(5)

,store_id,store_name,location
0,1,Store 1,New York
1,2,Store 2,Los Angeles
2,3,Store 3,New York
3,4,Store 4,New York
4,5,Store 5,Phoenix


In [6]:
transactions = pd.read_csv("data/transactions.csv")
transactions.head(5)

,transaction_id,customer_id,product_id,store_id,date,amount
0,1,35,8,12,2023-01-01,379.09
1,2,236,34,16,2023-01-02,18.48
2,3,334,50,20,2023-01-03,783.51
3,4,325,46,20,2023-01-04,333.82
4,5,332,89,6,2023-01-05,329.74


In [7]:
import sqlite3

In [9]:
con = sqlite3.connect('sales_data.db')

In [10]:
customers.to_sql(name="customers", con=con, if_exists="replace", index=False)
products.to_sql(name="products", con=con, if_exists="replace", index=False)
stores.to_sql(name="stores", con=con, if_exists="replace", index=False)
transactions.to_sql(name="transactions", con=con, if_exists="replace", index=False)

1000

In [20]:
tables = ['customers', 'products', 'stores', 'transactions']

table_info_dict = {}

for table in tables:
    query = f"PRAGMA table_info({table});"
    result = pd.read_sql_query(query, con)
    table_info_dict[table] = result

for table, df in table_info_dict.items():
    print(f"\nTable: {table}\n")
    print(df)


Table: customers

   cid           name     type  notnull dflt_value  pk
0    0    customer_id  INTEGER        0       None   0
1    1  customer_name     TEXT        0       None   0
2    2            age  INTEGER        0       None   0
3    3         gender     TEXT        0       None   0

Table: products

   cid          name     type  notnull dflt_value  pk
0    0    product_id  INTEGER        0       None   0
1    1  product_name     TEXT        0       None   0
2    2      category     TEXT        0       None   0
3    3         price     REAL        0       None   0

Table: stores

   cid        name     type  notnull dflt_value  pk
0    0    store_id  INTEGER        0       None   0
1    1  store_name     TEXT        0       None   0
2    2    location     TEXT        0       None   0

Table: transactions

   cid            name     type  notnull dflt_value  pk
0    0  transaction_id  INTEGER        0       None   0
1    1     customer_id  INTEGER        0       None   0
2   

In [26]:
query = '''SELECT transaction_id,COUNT(*) AS count
FROM transactions
GROUP BY transaction_id
HAVING COUNT(*) > 1'''
result = pd.read_sql_query(query, con)
result

,transaction_id,count


In [24]:
key_info = {}

for table in tables:
    primary_query = f"PRAGMA table_info({table});"
    primary_info = pd.read_sql_query(primary_query, con)

    primary_keys = primary_info[primary_info['pk'] == 1]['name'].tolist()

    foreign_query = f"PRAGMA table_info({table});"
    foreign_info = pd.read_sql_query(foreign_query, con) 

    key_info[table] = {
        'primary_keys': primary_keys,
        'foreign_keys': foreign_info.to_dict(orient='records') 
    }

for table, keys in key_info.items():
    print(f"\nTable: {table}")
    print(f"Primary Keys: {keys['primary_keys']}")
    print("Foreign Keys:")
    for fk in keys['foreign_keys']:
        print(fk)


Table: customers
Primary Keys: []
Foreign Keys:
{'cid': 0, 'name': 'customer_id', 'type': 'INTEGER', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 1, 'name': 'customer_name', 'type': 'TEXT', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 2, 'name': 'age', 'type': 'INTEGER', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 3, 'name': 'gender', 'type': 'TEXT', 'notnull': 0, 'dflt_value': None, 'pk': 0}

Table: products
Primary Keys: []
Foreign Keys:
{'cid': 0, 'name': 'product_id', 'type': 'INTEGER', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 1, 'name': 'product_name', 'type': 'TEXT', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 2, 'name': 'category', 'type': 'TEXT', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 3, 'name': 'price', 'type': 'REAL', 'notnull': 0, 'dflt_value': None, 'pk': 0}

Table: stores
Primary Keys: []
Foreign Keys:
{'cid': 0, 'name': 'store_id', 'type': 'INTEGER', 'notnull': 0, 'dflt_value': None, 'pk': 0}
{'cid': 1, 'name': 'store_name'

In [27]:
for table in tables:
    query = f"PRAGMA foreign_key_list({table});"
    result = pd.read_sql(query, con)
    print(f"Clés étrangères pour la table {table}:")
    print(result)

Clés étrangères pour la table customers:
Empty DataFrame
Columns: [id, seq, table, from, to, on_update, on_delete, match]
Index: []
Clés étrangères pour la table products:
Empty DataFrame
Columns: [id, seq, table, from, to, on_update, on_delete, match]
Index: []
Clés étrangères pour la table stores:
Empty DataFrame
Columns: [id, seq, table, from, to, on_update, on_delete, match]
Index: []
Clés étrangères pour la table transactions:
Empty DataFrame
Columns: [id, seq, table, from, to, on_update, on_delete, match]
Index: []


In [30]:
query = '''
SELECT date
FROM transactions
WHERE date(date) IS NOT NULL
AND date(date) IS NULL'''

invalid_dates = pd.read_sql(query, con)

if not invalid_dates.empty:
    print("Les valeurs suivantes ne sont pas des dates valides:")
    print(invalid_dates)
else:
    print("Toutes les dates sont valides.")

Toutes les dates sont valides.


In [31]:
query = ''' 
SELECT 
    SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS null_customer_id,
    SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS null_product_id,
    SUM(CASE WHEN store_id IS NULL THEN 1 ELSE 0 END) AS null_store_id
FROM transactions;
'''

result = pd.read_sql_query(query, con)
print("Nombre de valeurs nulles dans chaque colonne:")
print(result)

Nombre de valeurs nulles dans chaque colonne:
   null_customer_id  null_product_id  null_store_id
0                 0                0              0


In [40]:
# transactions made in NY store
query = ''' 
SELECT *
FROM stores
INNER JOIN transactions ON stores.store_id = transactions.store_id
WHERE stores.location = 'New York';  -- Assure-toi que le magasin est à NY
'''
result = pd.read_sql_query(query, con)
print(result)

     store_id store_name  location  transaction_id  customer_id  product_id  \
0           1    Store 1  New York               7          146          78   
1           1    Store 1  New York              12          500          81   
2           1    Store 1  New York              29           70          52   
3           1    Store 1  New York              51          400          62   
4           1    Store 1  New York              76          452          29   
..        ...        ...       ...             ...          ...         ...   
396        20   Store 20  New York             937          104          88   
397        20   Store 20  New York             941          370          74   
398        20   Store 20  New York             960          497           2   
399        20   Store 20  New York             978          292          95   
400        20   Store 20  New York             990          399          58   

     store_id        date  amount  
0           1  

In [45]:
# list of products names buying by customer_id 35
query = ''' 
SELECT product_name
FROM transactions
INNER JOIN products ON transactions.product_id = products.product_id
WHERE transactions.customer_id = 35
'''

result = pd.read_sql_query(query, con)
print(result)

  product_name
0    Product 8
1   Product 96


In [47]:
# list of different type category of available products
query = ''' 
SELECT category
FROM products
GROUP BY category
'''

result = pd.read_sql_query(query, con)
print(result)

      category
0        Books
1     Clothing
2  Electronics
3    Furniture


In [51]:
# find total amount of transactions for the store_id 12
query = ''' 
SELECT SUM(amount)
FROM transactions
WHERE store_id = 12
'''

result = pd.read_sql_query(query, con)
print(result)

   SUM(amount)
0     26796.29


In [55]:
# list of female customers 
query = ''' 
SELECT *
FROM customers
WHERE gender = "Female"
'''

result = pd.read_sql_query(query, con)
print(result)

     customer_id customer_name  age  gender
0              2    Customer 2   40  Female
1              7    Customer 7   55  Female
2              9    Customer 9   51  Female
3             10   Customer 10   22  Female
4             11   Customer 11   69  Female
..           ...           ...  ...     ...
226          490  Customer 490   53  Female
227          494  Customer 494   35  Female
228          496  Customer 496   59  Female
229          497  Customer 497   50  Female
230          500  Customer 500   69  Female

[231 rows x 4 columns]


In [62]:
# calculate mid amount of transaction per store
query = ''' 
SELECT AVG(amount), stores.store_name
FROM transactions
INNER JOIN stores ON transactions.store_id = stores.store_id
GROUP BY stores.store_name
'''

result = pd.read_sql_query(query, con)
print(result)

    AVG(amount) store_name
0    532.130000    Store 1
1    504.817115   Store 10
2    453.278361   Store 11
3    546.863061   Store 12
4    569.012857   Store 13
5    479.562745   Store 14
6    478.972679   Store 15
7    493.564167   Store 16
8    546.908519   Store 17
9    602.175526   Store 18
10   570.356531   Store 19
11   496.529592    Store 2
12   496.091786   Store 20
13   481.485789    Store 3
14   563.986364    Store 4
15   614.618200    Store 5
16   465.471190    Store 6
17   503.759412    Store 7
18   461.190196    Store 8
19   465.133200    Store 9


In [68]:
# find store which make the higher CA
query = ''' 
SELECT SUM(transactions.amount) AS total_CA, stores.store_name
FROM transactions
INNER JOIN stores ON transactions.store_id = stores.store_id
GROUP BY stores.store_name
ORDER BY total_CA DESC
LIMIT 1
'''

result = pd.read_sql_query(query, con)
print(result)

   total_CA store_name
0  30730.91    Store 5


In [69]:
# calculate total amount from each customers
query = ''' 
SELECT SUM(transactions.amount), customers.customer_name
FROM transactions
INNER JOIN customers ON transactions.customer_id = customers.customer_id
GROUP BY customers.customer_name
'''

result = pd.read_sql_query(query, con)
print(result)

     SUM(transactions.amount) customer_name
0                     2528.13    Customer 1
1                     1854.36   Customer 10
2                     2295.63  Customer 101
3                      682.86  Customer 102
4                      417.31  Customer 103
..                        ...           ...
434                   2541.37   Customer 95
435                    565.53   Customer 96
436                    403.17   Customer 97
437                   1101.91   Customer 98
438                   1943.84   Customer 99

[439 rows x 2 columns]


In [73]:
# find product with the most sellings (quantity)
query = ''' 
SELECT products.product_name, count(*) AS number_transactions
FROM products
INNER JOIN transactions ON transactions.product_id = products.product_id
GROUP BY products.product_name
ORDER BY number_transactions DESC
LIMIT 1
'''

result = pd.read_sql_query(query, con)
print(result)

  product_name  number_transactions
0   Product 45                   20


In [77]:
# list products categories and total amount of sellings per categories
query = ''' 
SELECT SUM(transactions.amount), products.category
FROM transactions
INNER JOIN products ON transactions.product_id = products.product_id
GROUP BY products.category
'''

result = pd.read_sql_query(query, con)
print(result)

   SUM(transactions.amount)     category
0                 151828.24        Books
1                 108614.35     Clothing
2                 125958.34  Electronics
3                 127514.47    Furniture


In [83]:
# customers who makes + 5 transactions
query = ''' 
SELECT customers.customer_id, COUNT (transactions.transaction_id)
FROM customers
INNER JOIN transactions ON customers.customer_id = transactions.customer_id
GROUP BY customers.customer_id
HAVING COUNT(transactions.transaction_id) > 5
'''

result = pd.read_sql_query(query, con)
print(result)

   customer_id  COUNT (transactions.transaction_id)
0           61                                    6
1          141                                    6
2          277                                    6


In [93]:
# find products which make over 1000 total sells
query = ''' 
SELECT products.product_name, SUM(transactions.amount)
FROM products
INNER JOIN transactions ON products.product_id = transactions.product_id
GROUP BY products.product_name
HAVING SUM(transactions.amount) > 1000
'''

result = pd.read_sql_query(query, con)
print(result)

   product_name  SUM(transactions.amount)
0     Product 1                   4334.66
1    Product 10                   2154.66
2   Product 100                   2560.02
3    Product 11                   4129.52
4    Product 12                   4236.36
..          ...                       ...
95   Product 95                   4382.96
96   Product 96                   6980.49
97   Product 97                   6947.89
98   Product 98                   4940.70
99   Product 99                   5780.27

[100 rows x 2 columns]


In [94]:
# list of stores with average amount transaction > 200 
query = ''' 
SELECT store_name, AVG(transactions.amount)
FROM stores
INNER JOIN transactions ON stores.store_id = transactions.store_id
GROUP BY stores.store_name
HAVING AVG(transactions.amount) > 200
'''

result = pd.read_sql_query(query, con)
print(result)

   store_name  AVG(transactions.amount)
0     Store 1                532.130000
1    Store 10                504.817115
2    Store 11                453.278361
3    Store 12                546.863061
4    Store 13                569.012857
5    Store 14                479.562745
6    Store 15                478.972679
7    Store 16                493.564167
8    Store 17                546.908519
9    Store 18                602.175526
10   Store 19                570.356531
11    Store 2                496.529592
12   Store 20                496.091786
13    Store 3                481.485789
14    Store 4                563.986364
15    Store 5                614.618200
16    Store 6                465.471190
17    Store 7                503.759412
18    Store 8                461.190196
19    Store 9                465.133200


In [102]:
# list books products, with over 500 sells
query = ''' 
SELECT products.product_name, SUM(transactions.amount)
FROM products
INNER JOIN transactions ON products.product_id = transactions.product_id
WHERE products.category = 'Books'
GROUP BY products.product_name
HAVING SUM(transactions.amount) > 500
'''

result = pd.read_sql_query(query, con)
print(result)

   product_name  SUM(transactions.amount)
0     Product 1                   4334.66
1    Product 14                   7558.51
2     Product 2                   5735.32
3    Product 20                   5097.30
4     Product 3                   5909.48
5    Product 31                   3538.25
6    Product 33                   6547.09
7    Product 39                   7603.02
8    Product 42                   7611.42
9    Product 47                   2999.79
10   Product 50                   4482.85
11   Product 52                   5492.80
12   Product 54                   7047.71
13   Product 55                   4650.90
14   Product 56                   1982.06
15    Product 6                   3147.49
16   Product 61                   7822.99
17   Product 62                   6269.32
18   Product 65                   3846.50
19   Product 69                   4718.06
20   Product 72                   5466.35
21   Product 73                   3885.76
22   Product 79                   

In [104]:
# find customers with over 1000 total expense
query = ''' 
SELECT customer_name, SUM(transactions.amount)
FROM customers
INNER JOIN transactions ON customers.customer_id = transactions.customer_id
GROUP BY customers.customer_name
HAVING SUM(transactions.amount) > 1000
'''

result = pd.read_sql_query(query, con)
print(result)

    customer_name  SUM(transactions.amount)
0      Customer 1                   2528.13
1     Customer 10                   1854.36
2    Customer 101                   2295.63
3    Customer 104                   1275.28
4    Customer 105                   1188.55
..            ...                       ...
217    Customer 9                   1039.92
218   Customer 93                   1385.94
219   Customer 95                   2541.37
220   Customer 98                   1101.91
221   Customer 99                   1943.84

[222 rows x 2 columns]


In [107]:
# extract year of the date transactions + count number of transactions per year
query = ''' 
SELECT strftime('%Y', date) AS year, COUNT(*) AS transaction_count
FROM transactions
GROUP BY year
ORDER BY year
'''

result = pd.read_sql_query(query, con)
print(result)

   year  transaction_count
0  2023                365
1  2024                366
2  2025                269


In [114]:
# concatenate name's client with his age
query = ''' 
SELECT customer_name || ' (' || age || ')' AS customer_info
FROM customers;
'''

result = pd.read_sql_query(query, con)
print(result)

         customer_info
0      Customer 1 (25)
1      Customer 2 (40)
2      Customer 3 (30)
3      Customer 4 (33)
4      Customer 5 (43)
..                 ...
495  Customer 496 (59)
496  Customer 497 (50)
497  Customer 498 (51)
498  Customer 499 (35)
499  Customer 500 (69)

[500 rows x 1 columns]


In [117]:
# replace all spaces with underscores in name products
query = ''' 
SELECT REPLACE(product_name, ' ', '_')
FROM products;
'''

result = pd.read_sql_query(query, con)
print(result)


   REPLACE(product_name, ' ', '_')
0                        Product_1
1                        Product_2
2                        Product_3
3                        Product_4
4                        Product_5
..                             ...
95                      Product_96
96                      Product_97
97                      Product_98
98                      Product_99
99                     Product_100

[100 rows x 1 columns]


In [118]:
# convert the store names to uppercase
query = ''' 
SELECT UPPER(store_name)
FROM stores
'''

result = pd.read_sql_query(query, con)
print(result)

   UPPER(store_name)
0            STORE 1
1            STORE 2
2            STORE 3
3            STORE 4
4            STORE 5
5            STORE 6
6            STORE 7
7            STORE 8
8            STORE 9
9           STORE 10
10          STORE 11
11          STORE 12
12          STORE 13
13          STORE 14
14          STORE 15
15          STORE 16
16          STORE 17
17          STORE 18
18          STORE 19
19          STORE 20


In [122]:
# join transaction table with store table + display the store name for each transaction 
query = ''' 
SELECT transactions.*, stores.store_name
FROM transactions
INNER JOIN stores ON stores.store_id = transactions.store_id
'''

result = pd.read_sql_query(query, con)
print(result)

     transaction_id  customer_id  product_id  store_id        date  amount  \
0                 1           35           8        12  2023-01-01  379.09   
1                 2          236          34        16  2023-01-02   18.48   
2                 3          334          50        20  2023-01-03  783.51   
3                 4          325          46        20  2023-01-04  333.82   
4                 5          332          89         6  2023-01-05  329.74   
..              ...          ...         ...       ...         ...     ...   
995             996          271          62         8  2025-09-22  268.09   
996             997          380           9         4  2025-09-23  829.86   
997             998           46          22        14  2025-09-24  603.12   
998             999            2          85         8  2025-09-25  882.43   
999            1000          253           4         8  2025-09-26  365.28   

    store_name  
0     Store 12  
1     Store 16  
2     Store 

In [123]:
# join transactions and products tables to get names of the products purchassed for each transactions
query = ''' 
SELECT transactions.*, products.product_name
FROM transactions
INNER JOIN products ON products.product_id = transactions.store_id
'''

result = pd.read_sql_query(query, con)
print(result)

     transaction_id  customer_id  product_id  store_id        date  amount  \
0                 1           35           8        12  2023-01-01  379.09   
1                 2          236          34        16  2023-01-02   18.48   
2                 3          334          50        20  2023-01-03  783.51   
3                 4          325          46        20  2023-01-04  333.82   
4                 5          332          89         6  2023-01-05  329.74   
..              ...          ...         ...       ...         ...     ...   
995             996          271          62         8  2025-09-22  268.09   
996             997          380           9         4  2025-09-23  829.86   
997             998           46          22        14  2025-09-24  603.12   
998             999            2          85         8  2025-09-25  882.43   
999            1000          253           4         8  2025-09-26  365.28   

    product_name  
0     Product_12  
1     Product_16  
2     

In [124]:
# join transactions/products/stores for display products purchassed, its store and its amount
query = ''' 
SELECT transactions.amount, products.product_name, stores.store_name
FROM transactions
INNER JOIN stores 
ON stores.store_id = transactions.store_id
INNER JOIN products
ON products.product_id = transactions.product_id
'''

result = pd.read_sql_query(query, con)
print(result)

     amount product_name store_name
0    379.09    Product_8   Store 12
1     18.48   Product_34   Store 16
2    783.51   Product_50   Store 20
3    333.82   Product_46   Store 20
4    329.74   Product_89    Store 6
..      ...          ...        ...
995  268.09   Product_62    Store 8
996  829.86    Product_9    Store 4
997  603.12   Product_22   Store 14
998  882.43   Product_85    Store 8
999  365.28    Product_4    Store 8

[1000 rows x 3 columns]


In [126]:
# list of transactions for costumers over 30 years 
query = ''' 
SELECT transactions.*
FROM transactions
INNER JOIN customers ON customers.customer_id = transactions.customer_id
WHERE age > 30
'''

result = pd.read_sql_query(query, con)
print(result)



     transaction_id  customer_id  product_id  store_id        date  amount
0               418            2          17        20  2024-02-22  556.39
1               666            2         100         3  2024-10-27  669.64
2               999            2          85         8  2025-09-25  882.43
3               446            4          93         3  2024-03-21  171.69
4               568            4          99         4  2024-07-21  847.98
..              ...          ...         ...       ...         ...     ...
781             345          498          60        15  2023-12-11  463.53
782             958          498          98         4  2025-08-15  283.84
783             127          499          34        10  2023-05-07  486.39
784              12          500          81         1  2023-01-12   11.31
785             604          500           5         5  2024-08-26  373.27

[786 rows x 6 columns]


In [129]:
# list of products buying by male customers
query = ''' 
SELECT products.*
FROM products
INNER JOIN transactions ON transactions.product_id = products.product_id
INNER JOIN customers ON customers.customer_id = transactions.customer_id
WHERE gender = "Male"
'''

result = pd.read_sql_query(query, con)
print(result)

     product_id product_name     category   price
0            35   Product 35  Electronics  451.66
1            83   Product 83  Electronics  350.40
2            88   Product 88     Clothing   10.63
3            89   Product 89     Clothing  125.60
4            11   Product 11     Clothing  267.75
..          ...          ...          ...     ...
545          86   Product 86        Books  254.77
546          60   Product 60     Clothing  402.04
547          74   Product 74  Electronics  404.55
548          98   Product 98        Books  412.89
549          34   Product 34  Electronics  392.02

[550 rows x 4 columns]


In [132]:
# find customers who made an amount transaction superior to the average transaction
query = ''' 
SELECT customers.*
FROM customers
INNER JOIN transactions ON transactions.customer_id = customers.customer_id
WHERE transactions.amount > (SELECT AVG(transactions.amount) FROM transactions)

'''

result = pd.read_sql_query(query, con)
print(result)

     customer_id customer_name  age  gender
0            334  Customer 334   58    Male
1            379  Customer 379   66  Female
2             42   Customer 42   63  Female
3             28   Customer 28   43  Female
4            453  Customer 453   38    Male
..           ...           ...  ...     ...
497          351  Customer 351   28    Male
498          189  Customer 189   47  Female
499          380  Customer 380   33  Female
500           46   Customer 46   30  Female
501            2    Customer 2   40  Female

[502 rows x 4 columns]


In [135]:
# list products with a price superior to the average price in their category 
query = ''' 
SELECT products.*
FROM products
WHERE price > (SELECT AVG(price) FROM products WHERE products.category = products.category)
'''

result = pd.read_sql_query(query, con)
print(result)

    product_id product_name     category   price
0            2    Product 2        Books  291.83
1            3    Product 3        Books  295.06
2            5    Product 5    Furniture  419.20
3            8    Product 8        Books  388.96
4            9    Product 9        Books  419.09
5           10   Product 10  Electronics  332.69
6           11   Product 11     Clothing  267.75
7           12   Product 12     Clothing  450.63
8           14   Product 14        Books  309.15
9           15   Product 15  Electronics  480.60
10          17   Product 17    Furniture  252.33
11          18   Product 18  Electronics  273.78
12          19   Product 19    Furniture  388.56
13          20   Product 20        Books  400.85
14          25   Product 25     Clothing  498.78
15          27   Product 27     Clothing  404.44
16          28   Product 28  Electronics  488.36
17          29   Product 29    Furniture  430.69
18          30   Product 30  Electronics  369.34
19          31   Pro

In [139]:
# use CTE for calculate total amount of transaction per customers, then list customers who made over 500 of transactions
query = ''' 
WITH TotalSpent AS (
    SELECT customer_id, SUM(amount) AS total_amount
    FROM transactions
    GROUP BY customer_id
)
SELECT customer_id, total_amount
FROM TotalSpent
WHERE total_amount > 500
'''

result = pd.read_sql_query(query, con)
print(result)

     customer_id  total_amount
0              1       2528.13
1              2       2108.46
2              4       1447.50
3              5        574.77
4              6       1934.12
..           ...           ...
338          494       2268.27
339          495       1814.12
340          496       2437.91
341          497       1821.58
342          498       1672.92

[343 rows x 2 columns]
